# Problem 2: Battery Packaging Simulation

### Skeleton / Notes

- setup:
  - 100 batteries per box
  - each battery ~ N(1, 0.05^2)
  - so total weight ~ Normal(mean=100, std=0.5)
  - choose N trials for Monte Carlo (like 200000)

- Monte Carlo:
  1. simulate N samples (N×100 normals)
  2. (a) prob(total > 101) = count/N
  3. (b) fee = max(X-100,0); conditional expectation = mean of fee when X>100

- Analytical:
  - use pdf/cdf for standard normal
  - (a) prob = 1 - Phi((101-100)/0.5)
  - (c) E[(X-100)+] = σφ(z) + (μ-100)(1-Φ(z))
  - (b) conditional = that / P(X>100)

- Finally: print MC vs analytic results This concludes the human portion/student skeleton. The following conversation with a language model (https://chatgpt.com/share/68d786aa-5a00-8009-a273-58771d66aad6) was used to generate the body of the code.

In [2]:
import numpy as np
from math import erf, sqrt, exp, pi

# --- setup ---
np.random.seed(42)                # reproducibility
n_per_box = 100
mu_each = 1.0
sd_each = 0.05

mu_total = n_per_box * mu_each
sd_total = np.sqrt(n_per_box) * sd_each   # = 0.5

N = 200_000  # number of Monte Carlo trials; bump this if you want tighter MC error

# --- helpers for standard normal ---
def phi(z):
    # standard normal pdf
    return (1.0 / sqrt(2*pi)) * exp(-0.5 * z*z)

def Phi(z):
    # standard normal cdf
    return 0.5 * (1.0 + erf(z / sqrt(2.0)))

# --- Monte Carlo simulation ---
# simulate N boxes: draw N x 100 normals with mean=1, sd=0.05 and sum to get totals
samples = np.random.normal(loc=mu_each, scale=sd_each, size=(N, n_per_box)).sum(axis=1)

# (a) estimate P(total > 101)
p_mc_gt_101 = np.mean(samples > 101.0)

# (b) conditional expectation of extra fee given X>100, where fee = max(X-100, 0)
fees = np.clip(samples - 100.0, 0.0, None)
mask_gt_100 = samples > 100.0
cond_fee_mc = fees[mask_gt_100].mean()

# --- Analytical values ---
# (a) analytic P(X>101)
z_101 = (101.0 - mu_total) / sd_total
p_analytic_gt_101 = 1.0 - Phi(z_101)

# (c) analytic E[(X-100)+] using the known formula
# For X ~ N(mu, sigma^2), k=100:
# E[(X-k)+] = sigma * phi((k - mu)/sigma) + (mu - k) * (1 - Phi((k - mu)/sigma))
k = 100.0
z_k = (k - mu_total) / sd_total
expected_positive_part = sd_total * phi(z_k) + (mu_total - k) * (1.0 - Phi(z_k))

# (b) analytic conditional expectation E[X-100 | X>100]
p_gt_100 = 1.0 - Phi(z_k)
cond_fee_analytic = expected_positive_part / p_gt_100

# --- print results side by side ---
print("Monte Carlo (N = {:,} trials)".format(N))
print(f"(a) P(X > 101) ≈ {p_mc_gt_101:.6f}")
print(f"(b) E[X - 100 | X > 100] ≈ ${cond_fee_mc:.6f}")

print("\nAnalytic")
print(f"(a) P(X > 101) = {p_analytic_gt_101:.6f}")
print(f"(b) E[X - 100 | X > 100] = ${cond_fee_analytic:.6f}")
print(f"(c) E[(X - 100)+] = ${expected_positive_part:.6f}")

# (optional) quick sanity checks:
# - MC estimate of E[(X-100)+] should be close to analytic when N is large
mc_pos_part = fees.mean()
print("\nSanity check: MC E[(X-100)+] ≈ ${:.6f}".format(mc_pos_part))

Monte Carlo (N = 200,000 trials)
(a) P(X > 101) ≈ 0.022915
(b) E[X - 100 | X > 100] ≈ $0.397774

Analytic
(a) P(X > 101) = 0.022750
(b) E[X - 100 | X > 100] = $0.398942
(c) E[(X - 100)+] = $0.199471

Sanity check: MC E[(X-100)+] ≈ $0.198947
